# Matrix Factorization Based Recommenders

Non-negative Matrix Factorization (NMF): This method factors the ratings matrix into two non-negative matrices, which can help with interpretability and handle non-negative data well.

Alternating Least Squares (ALS): ALS is an optimization algorithm that iteratively alternates between updating user factors and item factors. It's particularly popular in collaborative filtering recommender systems.

Probabilistic Matrix Factorization (PMF): PMF is a probabilistic approach to matrix factorization, where user-item interactions are modeled as Gaussian distributions. It aims to capture uncertainty in the ratings.

Weighted Alternating Least Squares (WALS): WALS is an extension of ALS that allows for incorporating side information or weights into the factorization process. It's useful when additional information about users or items is available.

Bayesian Personalized Ranking (BPR): BPR is a pairwise learning method that optimizes for ranking tasks rather than prediction accuracy. It's commonly used in recommendation systems where the goal is to recommend items that the user is likely to prefer over others.

Deep Learning-based Approaches: Various deep learning architectures, such as autoencoders, recurrent neural networks (RNNs), and convolutional neural networks (CNNs), have been applied to recommendation systems to learn latent representations of users and items.

In [1]:
#!pip install implicit

In [2]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from implicit.als import AlternatingLeastSquares

In [3]:
# Sample data (user, item, rating)
data = [
    {"user": "Alice", "item": "Guitar", "rating": 5},
    {"user": "Alice", "item": "Drums", "rating": 3},
    {"user": "Alice", "item": "Violin", "rating": 4},  
    {"user": "Bob", "item": "Guitar", "rating": 1},
    {"user": "Bob", "item": "TV", "rating": 5},
    {"user": "Bob", "item": "Radio", "rating": 4},
    {"user": "Bob", "item": "Laptop", "rating": 3}, 
    {"user": "Charlie", "item": "Guitar", "rating": 4},
    {"user": "Charlie", "item": "Piano", "rating": 5},
    {"user": "Charlie", "item": "Drums", "rating": 4},
    {"user": "Charlie", "item": "Microphone", "rating": 3}, 
    {"user": "David", "item": "Guitar", "rating": 2}, 
    {"user": "David", "item": "Violin", "rating": 4},  
    {"user": "David", "item": "Piano", "rating": 3}, 
    {"user": "David", "item": "Microphone", "rating": 2}, 
]

In [4]:
# Convert data to matrix (user x item)
users = sorted(set(d['user'] for d in data))
items = sorted(set(d['item'] for d in data))

user_idx = {u: i for i, u in enumerate(users)}
item_idx = {i: j for j, i in enumerate(items)}

ratings_matrix = np.zeros((len(users), len(items)))
for d in data:
    user_id = user_idx[d['user']]
    item_id = item_idx[d['item']]
    ratings_matrix[user_id, item_id] = d['rating']

### Singular Value Decomposition (SVD)

SVD decomposes a given data matrix into three matrices: U (orthogonal), Σ (diagonal), and VT (transpose of the conjugate transpose of U).

**User Item Interaction X**: represents the user-item interaction data. Each row corresponds to a user, and each column
corresponds to an item. E.g.,entry (i, j) = 4.2, it means that user i has given a rating of 4.2 for item j.

**Matrix U**: is an orthogonal matrix whose columns are the eigenvectors corresponding to the non-negative
singular values. Can be interpreted as a transformation of the original user-item interaction data into a new feature space with simpler structure.

**Diagonal Matrix Σ**: contains the singular values, which are non-negative scalars that represent the magnitude
of each principal component (column of `U`) in the transformed feature space. 

**Matrix VT**: is the transpose and conjugate transpose of the matrix $U$. It represents the weights of the original features (columns of `X`) in the transformed feature space.

In [5]:
# Singular Value Decomposition (SVD)
U, sigma, Vt = np.linalg.svd(ratings_matrix, full_matrices=False)

# Reconstruction of the ratings matrix
ratings_matrix_reconstructed = U @ np.diag(sigma) @ Vt

# Recommend top N items for a specific user
def recommend_items_svd(user_id, ratings_matrix, item_idx, N=3):
    user_ratings = ratings_matrix[user_id]
    unrated_items_idx = np.where(user_ratings == 0)[0]
    predicted_ratings = ratings_matrix_reconstructed[user_id]
    top_n_idx = np.argsort(predicted_ratings[unrated_items_idx])[::-1][:N]
    top_n_items = [list(item_idx.keys())[list(item_idx.values()).index(idx)] for idx in top_n_idx]
    return top_n_items

# Example: Recommend top 3 items for user 'Alice'
user_id = user_idx['Alice']
recommended_items = recommend_items_svd(user_id, ratings_matrix, item_idx)
print(f"Top 3 recommended items for user 'Alice': {recommended_items}")

Top 3 recommended items for user 'Alice': ['Piano', 'Microphone', 'Drums']


### Non-negative Matrix Factorization (NMF)

**Objective**: Aims to factorize a non-negative matrix $V$ into two non-negative matrices $W$ and $H$, such that $V≈WH$. Here, WW represents the user latent factors matrix, and $H$ represents the item latent factors matrix.

**Non-negativity Constraint**: Enforces non-negativity on the factor matrices $W$ and $H$. This constraint makes the resulting factorization more interpretable, as it leads to additive parts-based representations.

**Loss Function**: Minimizes the Frobenius norm or squared loss between the original matrix $V$ and its approximation $WH$, subject to the non-negativity constraints.

**Initialization**: Often uses random initialization or predefined initializations such as singular value decomposition (SVD) or random values drawn from a uniform distribution. The factor matrices are then iteratively updated to minimize the loss function.

**Algorithm**: The standard algorithm for NMF is multiplicative update rules, which iteratively updates the factor matrices $W$ and $H$ until convergence. Other algorithms include alternating non-negativity-constrained least squares (ANLS) and projected gradient descent.

**Number of Components**: The number of latent factors (components) $k$ is a hyperparameter that needs to be chosen beforehand. It controls the trade-off between approximation accuracy and model complexity.

**Interpretability**: NMF produces a parts-based representation, where each row of $W$ corresponds to a user's preferences over latent factors, and each column of HH corresponds to an item's features or characteristics in the latent space.

In [6]:
# Convert data to matrix (user x item)
users = sorted(set(d['user'] for d in data))
items = sorted(set(d['item'] for d in data))

user_idx = {u: i for i, u in enumerate(users)}
item_idx = {i: j for j, i in enumerate(items)}

user_ids = [user_idx[d['user']] for d in data]
item_ids = [item_idx[d['item']] for d in data]
ratings = [d['rating'] for d in data]

ratings_matrix = csr_matrix((ratings, (user_ids, item_ids)), shape=(len(users), len(items)))

In [7]:
# Non-negative Matrix Factorization (NMF)
nmf_model = NMF(n_components=2, init='random', random_state=42)
W_nmf = nmf_model.fit_transform(ratings_matrix)
H_nmf = nmf_model.components_
ratings_matrix_reconstructed_nmf = W_nmf.dot(H_nmf)

# Recommend top N items for a specific user using NMF
def recommend_items_nmf(user_id, ratings_matrix_reconstructed, item_idx, N=3):
    user_ratings = ratings_matrix_reconstructed[user_id]
    top_n_idx = np.argsort(user_ratings)[::-1][:N]
    top_n_items = [list(item_idx.keys())[list(item_idx.values()).index(idx)] for idx in top_n_idx]
    return top_n_items

# Example: Recommend top 3 items for user 'Alice' using NMF
user_id_alice = user_idx['Alice']
recommended_items_alice = recommend_items_nmf(user_id_alice, ratings_matrix_reconstructed_nmf, item_idx)
print(f"Top 3 recommended items for user 'Alice' using NMF: {recommended_items_alice}")


Top 3 recommended items for user 'Alice' using NMF: ['Guitar', 'Piano', 'Drums']


### Alternating Least Squares (ALS)

**Objective**: ALS aims to factorize a user-item ratings matrix $R$ into two low-rank matrices $U$ (user factors) and $V$ (item factors), such that $R≈UV^T$. Here, UU represents the latent factors of users, and VV represents the latent factors of items.

**Alternating Optimization**: ALS alternates between optimizing UU and VV while keeping the other matrix fixed. It iteratively updates one matrix while holding the other constant, and then switches roles.

**Least Squares Optimization**: ALS minimizes the least squares error between the observed ratings in $R$ and the predicted ratings $UV^T$. This is typically achieved through alternating least squares optimization, where one matrix is fixed and the other is optimized using least squares optimization.

**Regularization**: ALS often incorporates regularization terms to prevent overfitting. Regularization penalizes large values in the factor matrices, encouraging smaller and more generalizable solutions.

**Initialization**: ALS can be initialized with random values or with predefined initializations such as SVD. The initial values are then iteratively updated to minimize the least squares error.

**Number of Factors**: The number of latent factors is a hyperparameter that needs to be chosen beforehand. It controls the trade-off between approximation accuracy and model complexity.

**Implicit Feedback**: ALS can also be used with implicit feedback data, where the absence of a rating does not necessarily imply dislike. In such cases, ALS optimizes for ranking-based objectives, such as maximizing the likelihood of observed interactions.

Scalability: ALS is highly parallelizable and can be efficiently implemented on distributed computing platforms. This makes it scalable to large datasets and suitable for real-time recommendation systems.

In [8]:
# Alternating Least Squares (ALS)
als_model = AlternatingLeastSquares(factors=2, regularization=0.1, iterations=50, random_state=42)

# Fit the model
als_model.fit(ratings_matrix.T)

# Get the user and item factors
user_factors = als_model.user_factors
item_factors = als_model.item_factors

# Reconstruct the ratings matrix
ratings_matrix_reconstructed = user_factors.dot(item_factors.T)

# Recommend top N items for a specific user using ALS
def recommend_items_als(user_id, ratings_matrix_reconstructed, item_idx, N=3):
    user_ratings = ratings_matrix_reconstructed[user_id]
    top_n_idx = np.argsort(user_ratings)[::-1][:N]
    top_n_items = [list(item_idx.keys())[list(item_idx.values()).index(idx)] for idx in top_n_idx]
    return top_n_items

# Example: Recommend top 3 items for user 'Alice' using ALS
user_id_alice = user_idx['Alice']
recommended_items_alice = recommend_items_als(user_id_alice, ratings_matrix_reconstructed, item_idx)
print(f"Top 3 recommended items for user 'Alice' using ALS: {recommended_items_alice}")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/opt/homebrew/anaconda3/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 4.410743713378906e-05 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Top 3 recommended items for user 'Alice' using ALS: ['Laptop', 'Microphone', 'Drums']
